In [243]:
#Created: 15.02.2016
#Last modified: 18.02.2016
#               29.02.2016 add Corr and Dis cols
#               25.05.2016 updated input file and created truncated file for testing
#               24.05.2016 tested reading in file produced in birdhouse analogs process
#               05.09.2016 format test file for dc chart brush problem
#               09.09.2016 added test file that spans two months to test date handling
#                          change float format to 7 places in file save (float_format='%.7f')
#               13.09.2016 replace hard-coded num_analogues value with var name
#                          new test file with 11 analogues

In [1]:
import numpy as np
import pandas as pd
import collections
import os

In [43]:
num_analogues = 11 #20 #number of analogues searched for
num_cols = 3 #dateAnlg, Dis, Corr

In [44]:
#To read file produced in birdhouse
dfS = pd.DataFrame()
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/output_txt-0797016c-378e-11e6-91dd-41d8cd554993.txt'
#analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-c26ba398-711e-11e6-8d5a-fb3204a8dad1.txt'

#Analysis period > 1 year
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/analogs-5ee81d98-6fc9-11e6-a296-4d3fe2c8a29e.txt'
#analogs = 'http://birdhouse-lsce.extra.cea.fr:8090/wpsoutputs/flyingpigeon/analogs-73cd782c-74f4-11e6-bf5f-f73f2a3d7e35.txt'

#bbox x: [-80,50], y: [22.5,70], 11 analogues
analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-12f189be-79a7-11e6-b7f7-e7ff4fd8b248.txt'

#Analysis period spans two months (30.06.2013 to 02.07.2013)
#analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-7aafc4ec-74ea-11e6-b77d-eb5606a7fe9f.txt'

#one day only (03.01.2013)
#analogs = 'http://localhost:8090/wpsoutputs/flyingpigeon/analogs-9841a808-7504-11e6-b77d-eb5606a7fe9f.txt'

dfS = pd.read_csv(analogs, delimiter=r"\s+", index_col=0)
dfS.head()

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor2,cor3,cor4,cor5,cor6,cor7,cor8,cor9,cor10,cor11
date,,,,,,,,,,,,,,,,,,,,,
20130715,20140715,20140716,20140731,20140622,20140805,20140630,20140804,20140714,20140807,20140621,...,0.040807,0.040730,0.159989,-0.037593,-0.007528,-0.060790,-0.068464,-0.199696,0.202253,0.090306
20130716,20140622,20140716,20140621,20140715,20140805,20140620,20140623,20140808,20140804,20140618,...,0.135977,0.282605,0.151550,0.007938,0.341013,0.118046,0.010513,-0.011426,0.180286,0.150929
20130717,20140623,20140716,20140622,20140715,20140624,20140721,20140805,20140618,20140626,20140701,...,0.165520,0.255422,0.066167,0.108565,-0.005100,-0.105931,0.195877,-0.249135,0.090190,0.014971
20130718,20140623,20140622,20140624,20140716,20140626,20140715,20140717,20140625,20140618,20140727,...,0.424430,0.275362,0.157529,-0.088426,0.090526,0.107145,0.028597,0.293373,-0.023807,0.109294
20130719,20140622,20140623,20140725,20140624,20140726,20140625,20140724,20140626,20140630,20140723,...,0.574297,0.329109,0.361604,0.310559,0.171252,0.294422,0.009326,0.101402,0.315430,0.168408


In [4]:
# Create dataframe from analogues file
# Latest file: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat
# 2015: /home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2015.dat
# Note that last day in 2015 file is 20151225 (and NOT 20151231) because that is the last sunday of Dec 2015.

#dfS = pd.DataFrame()
#dfS = pd.read_csv('/home/estimr1/yiou/estimr1/NCEP/slpano-NA.analog30rms.1d.all1948-2016.dat', delimiter=' ', index_col=0)
#dfS.set_index('Date', inplace=True)

#dfS.head(5)

#dfS.index.name

In [45]:
dfS.tail(5)

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,...,cor2,cor3,cor4,cor5,cor6,cor7,cor8,cor9,cor10,cor11
date,,,,,,,,,,,,,,,,,,,,,
20141227,20130114,20130123,20130115,20130113,20130122,20130110,20130106,20131207,20130124,20130108,...,0.549200,0.543416,0.547768,0.346278,0.301867,0.516077,0.525985,0.483659,0.442039,0.362514
20141228,20130106,20130105,20130117,20130116,20130108,20130110,20130107,20130114,20130124,20130115,...,0.631067,0.644656,0.572964,0.602345,0.406935,0.542263,0.503951,0.585071,0.465256,0.499869
20141229,20130105,20130106,20130107,20130108,20130104,20130117,20140123,20130124,20130109,20130116,...,0.762899,0.663771,0.645353,0.568103,0.533461,0.494981,0.563119,0.290318,0.383496,0.604046
20141230,20130106,20130108,20130107,20130105,20130104,20130109,20130125,20131211,20131208,20140123,...,0.778094,0.754221,0.690132,0.609253,0.493589,0.589253,0.569573,0.462088,0.423612,0.417316
20141231,20131217,20130104,20130108,20130125,20140124,20130109,20130105,20131213,20131208,20131212,...,0.603565,0.557383,0.544596,0.492555,0.421007,0.502075,0.640764,0.403180,0.572354,0.493749


In [6]:
dfS.index

Int64Index([20130101, 20130102, 20130103, 20130104, 20130105, 20130106, 20130107, 20130108, 20130109, 20130110, 20130111, 20130112, 20130113, 20130114, 20130115, 20130116, 20130117, 20130118, 20130119, 20130120, 20130121, 20130122, 20130123, 20130124, 20130125, 20130126, 20130127, 20130128, 20130129, 20130130, 20130131, 20130201, 20130202, 20130203, 20130204, 20130205, 20130206, 20130207, 20130208, 20130209, 20130210, 20130211, 20130212, 20130213, 20130214, 20130215, 20130216, 20130217, 20130218, 20130219, 20130220, 20130221, 20130222, 20130223, 20130224, 20130225, 20130226, 20130227, 20130228, 20130301, 20130302, 20130303, 20130304, 20130305, 20130306, 20130307, 20130308, 20130309, 20130310, 20130311, 20130312, 20130313, 20130314, 20130315, 20130316, 20130317, 20130318, 20130319, 20130320, 20130321, 20130322, 20130323, 20130324, 20130325, 20130326, 20130327, 20130328, 20130329, 20130330, 20130331, 20130401, 20130402, 20130403, 20130404, 20130405, 20130406, 20130407, 20130408, 20130409

In [4]:
len(dfS.index)

730

In [5]:
#Chop off 2016 rows NO!!! there will still be 2016 dates found as analogues, which messes up js script
#dfS = dfS[dfS.index <= 20151231]

In [46]:
#SAVE DATE INDEX
#DF['id2'] = DF.index
dfS['dateRef'] = dfS.index
dfS['dateRef'].head(10)

date
20130715    20130715
20130716    20130716
20130717    20130717
20130718    20130718
20130719    20130719
20130720    20130720
20130721    20130721
20130722    20130722
20130723    20130723
20130724    20130724
Name: dateRef, dtype: int64

In [51]:
dfS.iloc[0:5, 0:num_analogues]

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,date.an11
date,,,,,,,,,,,
20130715,20140715,20140716,20140731,20140622,20140805,20140630,20140804,20140714,20140807,20140621,20140623
20130716,20140622,20140716,20140621,20140715,20140805,20140620,20140623,20140808,20140804,20140618,20140815
20130717,20140623,20140716,20140622,20140715,20140624,20140721,20140805,20140618,20140626,20140701,20140722
20130718,20140623,20140622,20140624,20140716,20140626,20140715,20140717,20140625,20140618,20140727,20140722
20130719,20140622,20140623,20140725,20140624,20140726,20140625,20140724,20140626,20140630,20140723,20140722


In [53]:
tmp = num_analogues + 2
dfS.iloc[0:5, 0:tmp]

,date.an1,date.an2,date.an3,date.an4,date.an5,date.an6,date.an7,date.an8,date.an9,date.an10,date.an11,dis1,dis2
date,,,,,,,,,,,,,
20130715,20140715,20140716,20140731,20140622,20140805,20140630,20140804,20140714,20140807,20140621,20140623,528.217,560.199
20130716,20140622,20140716,20140621,20140715,20140805,20140620,20140623,20140808,20140804,20140618,20140815,553.170,589.101
20130717,20140623,20140716,20140622,20140715,20140624,20140721,20140805,20140618,20140626,20140701,20140722,538.366,539.719
20130718,20140623,20140622,20140624,20140716,20140626,20140715,20140717,20140625,20140618,20140727,20140722,465.379,487.878
20130719,20140622,20140623,20140725,20140624,20140726,20140625,20140724,20140626,20140630,20140723,20140722,445.587,460.296


In [54]:
dfS.iloc[0:5, num_analogues:2*num_analogues]

,dis1,dis2,dis3,dis4,dis5,dis6,dis7,dis8,dis9,dis10,dis11
date,,,,,,,,,,,
20130715,528.217,560.199,561.790,562.651,581.428,596.056,611.714,612.736,616.273,617.265,617.849
20130716,553.170,589.101,614.791,621.192,638.320,639.902,650.524,652.808,655.144,665.755,667.242
20130717,538.366,539.719,542.755,596.082,611.484,619.836,631.386,631.402,640.034,648.576,651.132
20130718,465.379,487.878,511.392,521.341,555.264,556.591,563.674,564.890,571.153,574.745,578.558
20130719,445.587,460.296,507.203,515.246,523.311,551.821,554.138,554.828,557.850,567.952,572.929


In [55]:
dfS.iloc[0:5, 2*num_analogues:3*num_analogues]

,cor1,cor2,cor3,cor4,cor5,cor6,cor7,cor8,cor9,cor10,cor11
date,,,,,,,,,,,
20130715,0.158543,0.040807,0.040730,0.159989,-0.037593,-0.007528,-0.060790,-0.068464,-0.199696,0.202253,0.090306
20130716,0.248536,0.135977,0.282605,0.151550,0.007938,0.341013,0.118046,0.010513,-0.011426,0.180286,0.150929
20130717,0.323493,0.165520,0.255422,0.066167,0.108565,-0.005100,-0.105931,0.195877,-0.249135,0.090190,0.014971
20130718,0.482811,0.424430,0.275362,0.157529,-0.088426,0.090526,0.107145,0.028597,0.293373,-0.023807,0.109294
20130719,0.567457,0.574297,0.329109,0.361604,0.310559,0.171252,0.294422,0.009326,0.101402,0.315430,0.168408


In [56]:
#TRANSPOSE ANLG DATE COLS INTO SINGLE COL

#Define temporary df containing only anlg dates cols
df_anlg = dfS.iloc[:, 0:num_analogues]

#remove index name before transpose (stack)
df_anlg.index.name = ""

#transpose (stack)into single col
dateStack = df_anlg.stack()
dateStack.head(25)

20130715  date.an1     20140715
          date.an2     20140716
          date.an3     20140731
          date.an4     20140622
          date.an5     20140805
          date.an6     20140630
          date.an7     20140804
          date.an8     20140714
          date.an9     20140807
          date.an10    20140621
          date.an11    20140623
20130716  date.an1     20140622
          date.an2     20140716
          date.an3     20140621
          date.an4     20140715
          date.an5     20140805
          date.an6     20140620
          date.an7     20140623
          date.an8     20140808
          date.an9     20140804
          date.an10    20140618
          date.an11    20140815
20130717  date.an1     20140623
          date.an2     20140716
          date.an3     20140622
dtype: int64

In [57]:
#TRANSPOSE DIS COLS INTO SINGLE COL

#Define temporary df containing only dis cols
df_dis = dfS.iloc[:, num_analogues:2*num_analogues]

#remove index name before transpose (stack)
df_dis.index.name = ""

#transpose (stack) into single col and take absolute value
disStack = df_dis.stack().abs()
disStack.head(25)

20130715  dis1     528.217
          dis2     560.199
          dis3     561.790
          dis4     562.651
          dis5     581.428
          dis6     596.056
          dis7     611.714
          dis8     612.736
          dis9     616.273
          dis10    617.265
          dis11    617.849
20130716  dis1     553.170
          dis2     589.101
          dis3     614.791
          dis4     621.192
          dis5     638.320
          dis6     639.902
          dis7     650.524
          dis8     652.808
          dis9     655.144
          dis10    665.755
          dis11    667.242
20130717  dis1     538.366
          dis2     539.719
          dis3     542.755
dtype: float64

In [58]:
#TRANSPOSE CORR COLS INTO SINGLE COL

#Define temporary df containing only corr cols
df_corr = dfS.iloc[:, 2*num_analogues:3*num_analogues]

#remove index name before transpose (stack)
df_corr.index.name = ""

#transpose (stack) into single col
corrStack = df_corr.stack()
corrStack.head(25)

20130715  cor1     0.158543
          cor2     0.040807
          cor3     0.040730
          cor4     0.159989
          cor5    -0.037593
          cor6    -0.007528
          cor7    -0.060790
          cor8    -0.068464
          cor9    -0.199696
          cor10    0.202253
          cor11    0.090306
20130716  cor1     0.248536
          cor2     0.135977
          cor3     0.282605
          cor4     0.151550
          cor5     0.007938
          cor6     0.341013
          cor7     0.118046
          cor8     0.010513
          cor9    -0.011426
          cor10    0.180286
          cor11    0.150929
20130717  cor1     0.323493
          cor2     0.165520
          cor3     0.255422
dtype: float64

In [59]:
#BUILD NEW DF IN CORRECT FORMAT

#CREATE DF OF CORRECT DIMENSIONS (n x num_cols)
df_all = dfS.iloc[:, 0:num_cols]
#rename cols
df_all.columns = ['dateAnlg', 'Dis', 'Corr']

In [60]:
#dfS.index.tolist()
df_all.head(10)

,dateAnlg,Dis,Corr
,,,
20130715,20140715,20140716,20140731
20130716,20140622,20140716,20140621
20130717,20140623,20140716,20140622
20130718,20140623,20140622,20140624
20130719,20140622,20140623,20140725
20130720,20140725,20140724,20140723
20130721,20140725,20140724,20140723
20130722,20140626,20140725,20140625
20130723,20140626,20140806,20140725


In [61]:
df_all = df_all.set_index(dfS['dateRef'] )
df_all.head(10)

,dateAnlg,Dis,Corr
dateRef,,,
20130715,20140715,20140716,20140731
20130716,20140622,20140716,20140621
20130717,20140623,20140716,20140622
20130718,20140623,20140622,20140624
20130719,20140622,20140623,20140725
20130720,20140725,20140724,20140723
20130721,20140725,20140724,20140723
20130722,20140626,20140725,20140625
20130723,20140626,20140806,20140725


In [62]:
#REPLICATE INDEX COL 20 TIMES PER ROW

#http://stackoverflow.com/questions/26777832/replicating-rows-in-a-pandas-data-frame-by-a-column-value
#df2 = df.loc[np.repeat(df.index.values,df.n)]
df_all = df_all.loc[np.repeat(df_all.index.values,num_analogues)]

In [63]:
#REPLACE COLS WITH STACKED ANLG DATES, DIS AND CORR
df_all.iloc[:,0] = list(dateStack)
df_all.iloc[:,1] = list(disStack)
df_all.iloc[:,2] = list(corrStack)
df_all.head(25)

,dateAnlg,Dis,Corr
dateRef,,,
20130715,20140715,528.217,0.158543
20130715,20140716,560.199,0.040807
20130715,20140731,561.790,0.040730
20130715,20140622,562.651,0.159989
20130715,20140805,581.428,-0.037593
20130715,20140630,596.056,-0.007528
20130715,20140804,611.714,-0.060790
20130715,20140714,612.736,-0.068464
20130715,20140807,616.273,-0.199696


In [64]:
df_all.shape

(5885, 3)

In [66]:
df_all.tail(15)

,dateAnlg,Dis,Corr
dateRef,,,
20141230,20131211,917.602,0.569573
20141230,20131208,948.798,0.462088
20141230,20140123,954.568,0.423612
20141230,20131210,964.000,0.417316
20141231,20131217,794.941,0.613228
20141231,20130104,795.038,0.603565
20141231,20130108,844.628,0.557383
20141231,20130125,873.354,0.544596
20141231,20140124,876.450,0.492555


In [33]:
#dfS_select = df_all[df_all.index <= 19600101]

In [67]:
#SAVE TO CSV FILE

#df_all.to_csv('analogues_19480101_20151225.json', float_format='%.7f', sep='\t')
#df_all.to_csv('test2.json', float_format='%.7f', sep='\t')
#df_all.to_csv('test_gt1yr.json', float_format='%.7f', sep='\t')
#df_all.to_csv('test_gt1yr_birdhouse.json', float_format='%.7f', sep='\t')
df_all.to_csv('test_11anlgs.json', float_format='%.7f', sep='\t')
#df_all.to_csv('test_span2months.json', float_format='%.7f', sep='\t')
#df_all.to_csv('test_gt1yr_birdhouse_03Jan2013.json', float_format='%.7f', sep='\t')

#df_all.to_csv('analogues_19480101_20160520.json', float_format='%.7f', sep='\t')

#dfS_select.to_csv('analogues_select.json', float_format='%.7f', sep='\t')

In [77]:
pwd

u'/homel/cnangini/PROJECTS/analogues'

In [86]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [87]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [31]:
#check 1970-1979
df_check =  df_all[ (df_all['dateAnlg'] >= 19700101) & (df_all['dateAnlg'] <= 19791231) ]


#((df['column1']=='banana') & (df['colour']=='green'))

In [32]:
df_check.shape

(75935, 3)

In [33]:
df_check =  df_all[ (df_all['dateAnlg'] >= 20100101) & (df_all['dateAnlg'] <= 20191231) ]

In [34]:
df_check.shape

(44554, 3)

In [35]:
#cut file for testing purposes
df_select = df_all.iloc[19000:19100,:]

In [36]:
df_select.shape

(100, 3)

In [37]:
df_select.tail(10)

,dateAnlg,Dis,Corr
dateRef,,,
19500812,19840828,449.988840,0.546204
19500812,19590909,453.044659,0.612506
19500812,20030819,453.638210,0.559760
19500812,19970801,457.039066,0.597291
19500812,19980815,459.252587,0.534387
19500812,19990829,466.308358,0.580896
19500812,19510820,466.860763,0.727749
19500812,20130816,467.150673,0.623782
19500812,19990717,471.338490,0.571134


In [38]:
df_chunk = df_all.iloc[149000:149100,:]
df_chunk2 = df_all.iloc[106000:106100,:]

In [39]:
df_select = df_select.append(df_chunk)
df_select = df_select.append(df_chunk2)
#df_select = pd.concat([df_select,df_chunk])

In [40]:
df_select.shape

(300, 3)

In [41]:
#SAVE SELECTION TO TSV FILE

#df_select.to_csv('analogues_reformat_all_select.tsv', float_format='%.2f', sep='\t')

In [42]:
print df_all['Corr'].min(), df_all['Corr'].max()

-0.0773818303236 0.952985818969


In [43]:
print df_all['Dis'].min(), df_all['Dis'].max()

284.431297359 1369.41118104


In [44]:
df_select['Corr'].max()
df_select['Corr'].min()
print df_select['Corr'].min(), df_select['Corr'].max()

0.124791552993 0.821019151951


In [45]:
df_select['Dis'].max()
df_select['Dis'].min()
print df_select['Dis'].min(), df_select['Dis'].max()

350.217929418 696.281040107
